---
# **LAB 7 - Global memory (GMEM)**
---

# ▶️ CUDA setup

In [ ]:
!nvcc --version

In [ ]:
!nvidia-smi

## [GPU Compute Capability](https://developer.nvidia.com/cuda-gpus)

## NVCC Plugin for Jupyter notebook

*Usage*:


*   Load Extension `%load_ext nvcc_plugin`
*   Mark a cell to be treated as cuda cell
`%%cuda --name example.cu --compile false`

**NOTE**: The cell must contain either code or comments to be run successfully. It accepts 2 arguments. `-n | --name` - which is the name of either CUDA source or Header. The name parameter must have extension `.cu` or `.h`. Second argument -c | --compile; default value is false. The argument is a flag to specify if the cell will be compiled and run right away or not. It might be usefull if you're playing in the main function

*  We are ready to run CUDA C/C++ code right in your Notebook. For this we need explicitly say to the interpreter, that we want to use the extension by adding `%%cu` at the beginning of each cell with CUDA code. 




In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
%load_ext nvcc_plugin

## Bash and data setup

In [ ]:
#@title Bash setup
%%writefile /root/.bashrc

# If not running interactively, don't do anything
[ -z "$PS1" ] && return

# don't put duplicate lines in the history. See bash(1) for more options
# ... or force ignoredups and ignorespace
HISTCONTROL=ignoredups:ignorespace

# append to the history file, don't overwrite it
shopt -s histappend

# for setting history length see HISTSIZE and HISTFILESIZE in bash(1)
HISTSIZE=10000
HISTFILESIZE=20000

# check the window size after each command and, if necessary,
# update the values of LINES and COLUMNS.
shopt -s checkwinsize

# make less more friendly for non-text input files, see lesspipe(1)
[ -x /usr/bin/lesspipe ] && eval "$(SHELL=/bin/sh lesspipe)"

PS1='\[\033[01;34m\]\w\[\033[00m\]\$ '

# enable color support of ls and also add handy aliases
if [ -x /usr/bin/dircolors ]; then
    test -r ~/.dircolors && eval "$(dircolors -b ~/.dircolors)" || eval "$(dircolors -b)"
    alias ls='ls --color=auto'
    #alias dir='dir --color=auto'
    #alias vdir='vdir --color=auto'

    alias grep='grep --color=auto'
    alias fgrep='fgrep --color=auto'
    alias egrep='egrep --color=auto'
fi

# some more ls aliases
alias ll='ls -lF'
alias la='ls -A'
alias l='ls -CF'

# path setup
export PATH="./:/usr/local/cuda/bin:$PATH"

In [ ]:
!source /root/.bashrc

Clone GPUcomputing site on github...

In [ ]:
!git clone https://github.com/giulianogrossi/GPUcomputing.git

Define some paths...

In [ ]:
# path setup
!mkdir -p /content/GPUcomputing/lab7
%cd /content/GPUcomputing/lab7
!mkdir -p mems
!mkdir -p unified
!mkdir -p transpose
!mkdir -p struct

# ▶️ VS Code on Colab

In [ ]:
#@title Colab-ssh tunnel
#@markdown Execute this cell to open the ssh tunnel. Check [colab-ssh documentation](https://github.com/WassimBenzarti/colab-ssh) for more details.

# Install colab_ssh on google colab
!pip install colab_ssh --upgrade

from colab_ssh import launch_ssh_cloudflared, init_git_cloudflared
ssh_tunnel_password = "gpu" #@param {type: "string"}
launch_ssh_cloudflared(password=ssh_tunnel_password)

# Optional: if you want to clone a Github or Gitlab repository
repository_url="https://github.com/giulianogrossi/GPUcomputing" #@param {type: "string"}
init_git_cloudflared(repository_url)

# ▶️ DeviceQuery

In [ ]:
# DeviceQuery dell'attuale device (su Colab!)
!nvcc /content/GPUcomputing/utils/deviceQuery.cu -o deviceQuery
!./deviceQuery

# ✅ Static, pinned, zero-copy  memory

**Pinned memory**

An example of using CUDA's memory copy API to transfer data to and from the device. In this case, `cudaMalloc` is used to allocate memory on the GPU and `cudaMemcpy` is used to transfer the contents of host memory to an array allocated using `cudaMalloc`. Host memory is allocated using `cudaMallocHost` to create a page-locked host array.

In [ ]:
%%writefile mems/pinMemTransfer.cu

#include "../../utils/common.h"
#include <cuda_runtime.h>
#include <stdio.h>


int main(int argc, char **argv) {
  // set up device
  int dev = 0;
  CHECK(cudaSetDevice(dev));

  // memory size
  unsigned int isize = 1 << 24;
  unsigned int nbytes = isize * sizeof(float);

  // get device information
  cudaDeviceProp deviceProp;
  CHECK(cudaGetDeviceProperties(&deviceProp, dev));

  if (!deviceProp.canMapHostMemory) {
      printf("Device %d does not support mapping CPU host memory!\n", dev);
      CHECK(cudaDeviceReset());
      exit(EXIT_SUCCESS);
  }

  printf("%s starting at ", argv[0]);
  printf("device %d: %s memory size %d nbyte %5.2fMB canMap %d\n", dev,
          deviceProp.name, isize, nbytes / (1024.0f * 1024.0f),
          deviceProp.canMapHostMemory);

  float *h_a;
  // allocate the host memory   
  //h_a = (float *)malloc(nbytes);

  // allocate pinned host memory
  CHECK(cudaMallocHost ((float **)&h_a, nbytes));

  // allocate device memory
  float *d_a;
  CHECK(cudaMalloc((float **)&d_a, nbytes));

  for (int i = 0; i < isize; i++) 
    h_a[i] = 100.10f;

  // transfer data from the host to the device
  CHECK(cudaMemcpy(d_a, h_a, nbytes, cudaMemcpyHostToDevice));

  // transfer data from the device to the host
  CHECK(cudaMemcpy(h_a, d_a, nbytes, cudaMemcpyDeviceToHost));

  // free memory
  CHECK(cudaFree(d_a));
  CHECK(cudaFreeHost(h_a));

  // reset device
  CHECK(cudaDeviceReset());
  return EXIT_SUCCESS;
}


In [ ]:
!nvcc -O3 -arch=sm_37 mems/pinMemTransfer.cu -o pinMemTransfer
!nvprof ./pinMemTransfer


# ✅ Unified memory


In [ ]:
%%writefile unified/sumMatrix.cu

#include <stdio.h>
#include "../../utils/common.h"

void initialData(float *ip, const int size) {
  int i;

  for (i = 0; i < size; i++)
    ip[i] = (float)( rand() & 0xFF ) / 10.0f;
  return;
}

void sumMatrixOnHost(float *A, float *B, float *C, const int nx, const int ny) {
  float *ia = A;
  float *ib = B;
  float *ic = C;

  for (int iy = 0; iy < ny; iy++) {
    for (int ix = 0; ix < nx; ix++)
      ic[ix] = ia[ix] + ib[ix];

    ia += nx;
    ib += nx;
    ic += nx;
  }
  return;
}

void checkResult(float *hostRef, float *gpuRef, const int N) {
  double epsilon = 1.0E-8;
  bool match = 1;

  for (int i = 0; i < N; i++) {
    if (abs(hostRef[i] - gpuRef[i]) > epsilon) {
      match = 0;
      printf("host %f gpu %f\n", hostRef[i], gpuRef[i]);
      break;
    }
  }

  if (!match)
    printf("Arrays do not match.\n\n");
}

// grid 2D block 2D
__global__ void sumMatrixGPU(float *MatA, float *MatB, float *MatC, int nx, int ny) {
  unsigned int ix = threadIdx.x + blockIdx.x * blockDim.x;
  unsigned int iy = threadIdx.y + blockIdx.y * blockDim.y;
  unsigned int idx = iy * nx + ix;

  if (ix < nx && iy < ny)
    MatC[idx] = MatA[idx] + MatB[idx];
}

int main(int argc, char **argv) {
    printf("%s Starting ", argv[0]);

    // set up device
    int dev = 0;
    cudaDeviceProp deviceProp;
    CHECK(cudaGetDeviceProperties(&deviceProp, dev));
    printf("using Device %d: %s\n", dev, deviceProp.name);
    CHECK(cudaSetDevice(dev));

    // set up data size of matrix
    int nx, ny;
    int ishift = 12;

    if  (argc > 1) ishift = atoi(argv[1]);

    nx = ny = 1 << ishift;

    int nxy = nx * ny;
    int nBytes = nxy * sizeof(float);
    printf("Matrix size: nx %d ny %d\n", nx, ny);

    // malloc host memory
    float *h_A, *h_B, *hostRef, *gpuRef;
    h_A = (float *)malloc(nBytes);
    h_B = (float *)malloc(nBytes);
    hostRef = (float *)malloc(nBytes);
    gpuRef = (float *)malloc(nBytes);

    // initialize data at host side
    double iStart = seconds();
    initialData(h_A, nxy);
    initialData(h_B, nxy);
    double iElaps = seconds() - iStart;

    printf("initialization: \t %f sec\n", iElaps);

    memset(hostRef, 0, nBytes);
    memset(gpuRef, 0, nBytes);

    // add matrix at host side for result checks
    iStart = seconds();
    sumMatrixOnHost(h_A, h_B, hostRef, nx, ny);
    iElaps = seconds() - iStart;
    printf("sumMatrix on host:\t %f sec\n", iElaps);

    // malloc device global memory
    float *d_MatA, *d_MatB, *d_MatC;
    CHECK(cudaMalloc((void **)&d_MatA, nBytes));
    CHECK(cudaMalloc((void **)&d_MatB, nBytes));
    CHECK(cudaMalloc((void **)&d_MatC, nBytes));

    // invoke kernel at host side
    int dimx = 32;
    int dimy = 32;
    dim3 block(dimx, dimy);
    dim3 grid((nx + block.x - 1) / block.x, (ny + block.y - 1) / block.y);

    // init device data to 0.0f, then warm-up kernel to obtain accurate timing
    // result
    CHECK(cudaMemset(d_MatA, 0.0f, nBytes));
    CHECK(cudaMemset(d_MatB, 0.0f, nBytes));
    sumMatrixGPU<<<grid, block>>>(d_MatA, d_MatB, d_MatC, 1, 1);


    // transfer data from host to device
    CHECK(cudaMemcpy(d_MatA, h_A, nBytes, cudaMemcpyHostToDevice));
    CHECK(cudaMemcpy(d_MatB, h_B, nBytes, cudaMemcpyHostToDevice));

    iStart =  seconds();
    sumMatrixGPU<<<grid, block>>>(d_MatA, d_MatB, d_MatC, nx, ny);

    CHECK(cudaDeviceSynchronize());
    iElaps = seconds() - iStart;
    printf("sumMatrix on gpu :\t %f sec <<<(%d,%d), (%d,%d)>>> \n", iElaps,
            grid.x, grid.y, block.x, block.y);

    CHECK(cudaMemcpy(gpuRef, d_MatC, nBytes, cudaMemcpyDeviceToHost));

    // check kernel error
    CHECK(cudaGetLastError());

    // check device results
    checkResult(hostRef, gpuRef, nxy);

    // free device global memory
    CHECK(cudaFree(d_MatA));
    CHECK(cudaFree(d_MatB));
    CHECK(cudaFree(d_MatC));

    // free host memory
    free(h_A);
    free(h_B);
    free(hostRef);
    free(gpuRef);

    // reset device
    CHECK(cudaDeviceReset());

    return (0);
}


In [ ]:
# Compilazione ed esecuzione

!nvcc -arch=sm_37 unified/sumMatrix.cu  -o sumMatrix
!./sumMatrix 14

In [ ]:
# profilazione

!nvprof ./sumMatrix 14

# 🔴 TODO

In [ ]:
%%writefile unified/sumMatrixUni.cu

#include "../../utils/common.h"

void initialData(float *ip, const int size) {
  int i;

  for (i = 0; i < size; i++)
    ip[i] = (float)( rand() & 0xFF ) / 10.0f;
  return;
}

void sumMatrixOnHost(float *A, float *B, float *C, const int nx, const int ny) {
  float *ia = A;
  float *ib = B;
  float *ic = C;

  for (int iy = 0; iy < ny; iy++) {
    for (int ix = 0; ix < nx; ix++)
      ic[ix] = ia[ix] + ib[ix];

    ia += nx;
    ib += nx;
    ic += nx;
  }
  return;
}

void checkResult(float *hostRef, float *gpuRef, const int N) {
  double epsilon = 1.0E-8;
  bool match = 1;

  for (int i = 0; i < N; i++) {
    if (abs(hostRef[i] - gpuRef[i]) > epsilon) {
      match = 0;
      printf("host %f gpu %f\n", hostRef[i], gpuRef[i]);
      break;
    }
  }

  if (!match)
    printf("Arrays do not match.\n\n");
}

// grid 2D block 2D
__global__ void sumMatrixGPU(float *MatA, float *MatB, float *MatC, int nx, int ny) {
  unsigned int ix = threadIdx.x + blockIdx.x * blockDim.x;
  unsigned int iy = threadIdx.y + blockIdx.y * blockDim.y;
  unsigned int idx = iy * nx + ix;

  if (ix < nx && iy < ny)
    MatC[idx] = MatA[idx] + MatB[idx];
}

int main(int argc, char **argv) {
  printf("%s Starting ", argv[0]);

  // set up device
  int dev = 0;
  cudaDeviceProp deviceProp;
  CHECK(cudaGetDeviceProperties(&deviceProp, dev));
  printf("using Device %d: %s\n", dev, deviceProp.name);
  CHECK(cudaSetDevice(dev));

  // set up data size of matrix
  int nx, ny;
  int ishift = 14;

  if  (argc > 1) ishift = atoi(argv[1]);

  nx = ny = 1 << ishift;

  int nxy = nx * ny;
  int nBytes = nxy * sizeof(float);
  printf("Matrix size: nx %d ny %d\n", nx, ny);

  // malloc host memory
  float *A, *B, *hostRef, *gpuRef;
  CHECK(cudaMallocManaged((void **)&A, nBytes));
  CHECK(cudaMallocManaged((void **)&B, nBytes));
  CHECK(cudaMallocManaged((void **)&gpuRef,  nBytes);  );
  CHECK(cudaMallocManaged((void **)&hostRef, nBytes););

  // initialize data at host side
  double iStart = seconds();
  initialData(A, nxy);
  initialData(B, nxy);
  double iElaps = seconds() - iStart;
  printf("initialization: \t %f sec\n", iElaps);

  memset(hostRef, 0, nBytes);
  memset(gpuRef, 0, nBytes);

  // add matrix at host side for result checks
  iStart = seconds();
  sumMatrixOnHost(A, B, hostRef, nx, ny);
  iElaps = seconds() - iStart;
  printf("sumMatrix on host:\t %f sec\n", iElaps);

  // invoke kernel at host side
  int dimx = 32;
  int dimy = 32;
  dim3 block(dimx, dimy);
  dim3 grid((nx + block.x - 1) / block.x, (ny + block.y - 1) / block.y);

  // warm-up kernel, with unified memory all pages will migrate from host to device
  sumMatrixGPU<<<grid, block>>>(A, B, gpuRef, 1, 1);

  // after warm-up, time with unified memory
  iStart = seconds();

  sumMatrixGPU<<<grid, block>>>(A, B, gpuRef, nx, ny);

  CHECK(cudaDeviceSynchronize());
  iElaps = seconds() - iStart;
  printf("sumMatrix on gpu :\t %f sec <<<(%d,%d), (%d,%d)>>> \n", iElaps,
          grid.x, grid.y, block.x, block.y);

  // check kernel error
  CHECK(cudaGetLastError());

  // check device results
  checkResult(hostRef, gpuRef, nxy);

  // free device global memory
  CHECK(cudaFree(A));
  CHECK(cudaFree(B));
  CHECK(cudaFree(hostRef));
  CHECK(cudaFree(gpuRef));

  // reset device
  CHECK(cudaDeviceReset());

  return (0);
}


In [ ]:
# Compilazione ed esecuzione

!nvcc -arch=sm_37 unified/sumMatrixUni.cu  -o sumMatrixUni
!./sumMatrixUni 14

In [ ]:
# profilazione

!nvprof ./sumMatrixUni

In [ ]:
%%writefile unified_mem/sumMatrixGPUManual.cu

#include "../../utils/common.h"

void initialData(float *ip, const int size) {
  int i;

  for (i = 0; i < size; i++)
    ip[i] = (float)( rand() & 0xFF ) / 10.0f;
  return;
}

void sumMatrixOnHost(float *A, float *B, float *C, const int nx, const int ny) {
  float *ia = A;
  float *ib = B;
  float *ic = C;

  for (int iy = 0; iy < ny; iy++) {
    for (int ix = 0; ix < nx; ix++)
      ic[ix] = ia[ix] + ib[ix];

    ia += nx;
    ib += nx;
    ic += nx;
  }
  return;
}

void checkResult(float *hostRef, float *gpuRef, const int N) {
  double epsilon = 1.0E-8;
  bool match = 1;

  for (int i = 0; i < N; i++) {
    if (abs(hostRef[i] - gpuRef[i]) > epsilon) {
      match = 0;
      printf("host %f gpu %f\n", hostRef[i], gpuRef[i]);
      break;
    }
  }

  if (!match)
    printf("Arrays do not match.\n\n");
}

// grid 2D block 2D
__global__ void sumMatrixGPU(float *MatA, float *MatB, float *MatC, int nx, int ny) {
  unsigned int ix = threadIdx.x + blockIdx.x * blockDim.x;
  unsigned int iy = threadIdx.y + blockIdx.y * blockDim.y;
  unsigned int idx = iy * nx + ix;

  if (ix < nx && iy < ny)
    MatC[idx] = MatA[idx] + MatB[idx];
}

int main(int argc, char **argv) {
    printf("%s Starting ", argv[0]);

    // set up device
    int dev = 0;
    cudaDeviceProp deviceProp;
    CHECK(cudaGetDeviceProperties(&deviceProp, dev));
    printf("using Device %d: %s\n", dev, deviceProp.name);
    CHECK(cudaSetDevice(dev));

    // set up data size of matrix
    int nx, ny;
    int ishift = 12;

    if  (argc > 1) ishift = atoi(argv[1]);

    nx = ny = 1 << ishift;

    int nxy = nx * ny;
    int nBytes = nxy * sizeof(float);
    printf("Matrix size: nx %d ny %d\n", nx, ny);

    // malloc host memory
    float *h_A, *h_B, *hostRef, *gpuRef;
    h_A = (float *)malloc(nBytes);
    h_B = (float *)malloc(nBytes);
    hostRef = (float *)malloc(nBytes);
    gpuRef = (float *)malloc(nBytes);

    // initialize data at host side
    double iStart = seconds();
    initialData(h_A, nxy);
    initialData(h_B, nxy);
    double iElaps = seconds() - iStart;

    printf("initialization: \t %f sec\n", iElaps);

    memset(hostRef, 0, nBytes);
    memset(gpuRef, 0, nBytes);

    // add matrix at host side for result checks
    iStart = seconds();
    sumMatrixOnHost(h_A, h_B, hostRef, nx, ny);
    iElaps = seconds() - iStart;
    printf("sumMatrix on host:\t %f sec\n", iElaps);

    // malloc device global memory
    float *d_MatA, *d_MatB, *d_MatC;
    CHECK(cudaMalloc((void **)&d_MatA, nBytes));
    CHECK(cudaMalloc((void **)&d_MatB, nBytes));
    CHECK(cudaMalloc((void **)&d_MatC, nBytes));

    // invoke kernel at host side
    int dimx = 32;
    int dimy = 32;
    dim3 block(dimx, dimy);
    dim3 grid((nx + block.x - 1) / block.x, (ny + block.y - 1) / block.y);

    // init device data to 0.0f, then warm-up kernel to obtain accurate timing
    // result
    CHECK(cudaMemset(d_MatA, 0.0f, nBytes));
    CHECK(cudaMemset(d_MatB, 0.0f, nBytes));
    sumMatrixGPU<<<grid, block>>>(d_MatA, d_MatB, d_MatC, 1, 1);


    // transfer data from host to device
    CHECK(cudaMemcpy(d_MatA, h_A, nBytes, cudaMemcpyHostToDevice));
    CHECK(cudaMemcpy(d_MatB, h_B, nBytes, cudaMemcpyHostToDevice));

    iStart =  seconds();
    sumMatrixGPU<<<grid, block>>>(d_MatA, d_MatB, d_MatC, nx, ny);

    CHECK(cudaDeviceSynchronize());
    iElaps = seconds() - iStart;
    printf("sumMatrix on gpu :\t %f sec <<<(%d,%d), (%d,%d)>>> \n", iElaps,
            grid.x, grid.y, block.x, block.y);

    CHECK(cudaMemcpy(gpuRef, d_MatC, nBytes, cudaMemcpyDeviceToHost));

    // check kernel error
    CHECK(cudaGetLastError());

    // check device results
    checkResult(hostRef, gpuRef, nxy);

    // free device global memory
    CHECK(cudaFree(d_MatA));
    CHECK(cudaFree(d_MatB));
    CHECK(cudaFree(d_MatC));

    // free host memory
    free(h_A);
    free(h_B);
    free(hostRef);
    free(gpuRef);

    // reset device
    CHECK(cudaDeviceReset());

    return (0);
}


In [ ]:
# Compilazione ed esecuzione

!nvcc -arch=sm_60 sumMatrixGPUManual.cu  -o sumMatrixGPUManual
!sumMatrixGPUManual 14

In [ ]:
# profilazione (senza unified memory - dà errore)

!nvprof --unified-memory-profiling off ./sumMatrixGPUManual

# ✅ SoA vs AoS structs

In [ ]:
%%writefile struct/SoA.cu

#include <stdint.h>
#include "../../utils/common.h"

#define N 1<<24
#define blocksize 1<<7

struct SoA {
	uint8_t r[N];
	uint8_t g[N];
	uint8_t b[N];
};


void initialize(SoA*, int);
void checkResult(SoA*, SoA*, int);

/*
 * Riscala l'immagine al valore massimo [max] fissato
 */
__global__ void rescaleImg(SoA *img, const int max, const int n) {
	unsigned int i = blockIdx.x * blockDim.x + threadIdx.x;
	if (i < n) {
		float r,g,b;
		SoA *tmp = img;
		r = max * (float)tmp->r[i]/255.0f;
		img->r[i] = (uint8_t)r;
		g = max * (float)tmp->g[i]/255.0f;
		img->g[i] = (uint8_t)g;
		b = max * (float)tmp->b[i]/255.0f;
		img->b[i] = (uint8_t)b;
	}
}

/*
 * cancella un piano dell'immagine [plane = 'r' o 'g' o 'b'] fissato
 */
__global__ void deletePlane(SoA *img, const char plane, const int n) {
	unsigned int i = blockIdx.x * blockDim.x + threadIdx.x;
	if (i < n) {
		switch (plane) {
		case 'r':
			img->r[i] = 0;
			break;
		case 'g':
			img->g[i] = 0;
			break;
		case 'b':
			img->b[i] = 0;
			break;
		}
	}
}

/*
 * setup device
 */
__global__ void warmup(SoA *img, const int max, const int n) {
	unsigned int i = blockIdx.x * blockDim.x + threadIdx.x;
	if (i < n) {
		float r,g,b;
		SoA *tmp = img;
		r = max * (float)tmp->r[i]/255.0f;
		img->r[i] = (uint8_t)r;
		g = max * (float)tmp->g[i]/255.0f;
		img->g[i] = (uint8_t)g;
		b = max * (float)tmp->b[i]/255.0f;
		img->b[i] = (uint8_t)b;
	}
}

/*
 * Legge da stdin quale kernel eseguire: 0 per rescaleImg, 1 per deletePlane
 */
int main(int argc, char **argv) {
	// set up device
	int dev = 0;
	cudaDeviceProp deviceProp;
	CHECK(cudaGetDeviceProperties(&deviceProp, dev));
	printf("%s test SoA at ", argv[0]);
	printf("device %d: %s \n", dev, deviceProp.name);
	CHECK(cudaSetDevice(dev));

	// scelta del kernel da eseguire
	int kernel = 0;
	if (argc > 1) kernel = atoi(argv[1]);

	// allocate host memory
	size_t nBytes = sizeof(SoA);
	SoA *img = (SoA *)malloc(nBytes);
	SoA *new_img = (SoA *)malloc(nBytes);

	// initialize host array
	initialize(img, N);

	// allocate device memory
	int n_elem = N;
	SoA *d_img;
	CHECK(cudaMalloc((void**)&d_img, nBytes));

	// copy data from host to device
	CHECK(cudaMemcpy(d_img, img, nBytes, cudaMemcpyHostToDevice));

	// definizione max
	int max = 128;
	if (argc > 2) max = atoi(argv[2]);

	// configurazione per esecuzione
	dim3 block (blocksize, 1);
	dim3 grid  ((n_elem + block.x - 1) / block.x, 1);

	// kernel 1: warmup
	double iStart = seconds();
	warmup<<<1, 32>>>(d_img, max, 32);
	CHECK(cudaDeviceSynchronize());
	double iElaps = seconds() - iStart;
	printf("warmup<<< 1, 32 >>> elapsed %f sec\n",iElaps);
	CHECK(cudaGetLastError());

	// kernel 2 rescaleImg o deletePlane
	iStart = seconds();
	if (kernel == 0)
		rescaleImg<<<grid, block>>>(d_img, max, n_elem);
	else
		deletePlane<<<grid, block>>>(d_img, 'r', n_elem);
	CHECK(cudaDeviceSynchronize());
	iElaps = seconds() - iStart;
	printf("rescaleImg <<< %3d, %3d >>> elapsed %f sec\n", grid.x, block.x, iElaps);
	CHECK(cudaMemcpy(new_img, d_img, nBytes, cudaMemcpyDeviceToHost));
	CHECK(cudaGetLastError());

	//checkResult(img, new_img, n_elem);

	// free memories both host and device
	CHECK(cudaFree(d_img));
	free(img);
	free(new_img);

	// reset device
	CHECK(cudaDeviceReset());
	return EXIT_SUCCESS;
}

void initialize(SoA *img,  int size) {
	for (int i = 0; i < size; i++) {
		img->r[i] = rand() % 256;
		img->g[i] = rand() % 256;
		img->b[i] = rand() % 256;
	}
	return;
}

void checkResult(SoA *img, SoA *new_img, int n_elem) {
	for (int i = 0; i < n_elem; i+=1000)
		printf("img[%d] = (%d,%d,%d) -- new_img[%d] = (%d,%d,%d)\n",
				i,img->r[i],img->g[i],img->b[i],i,new_img->r[i],new_img->g[i],new_img->b[i]);
	return;
}


void transposeHost(float *out, float *in, const int nx, const int ny) {
	for (int iy = 0; iy < ny; ++iy) {
		for (int ix = 0; ix < nx; ++ix) {
			out[ix * ny + iy] = in[iy * nx + ix];
		}
	}
}



In [ ]:
# Compilazione ed esecuzione
!nvcc -arch=sm_37  struct/SoA.cu -o SoA
!./SoA

# 🔴 TODO

In [ ]:
%%writefile struct/AoS.cu

#include <stdint.h>
#include "../../utils/common.h"

#define N 1<<24
#define blocksize 128

struct AoS {
	uint8_t r;
	uint8_t g;
	uint8_t b;
};

void initialize(AoS *, int);
void checkResult(AoS *, AoS *, int);

/*
 * Riscala l'immagine al valore massimo [max] fissato
 */
__global__ void rescaleImg(AoS *img, const int max, const int n) {
	unsigned int i = blockIdx.x * blockDim.x + threadIdx.x;
	if (i < n) {
		float r,g,b;
		AoS tmp = img[i];
		r = max * (float)tmp.r/255.0f;
		tmp.r = (uint8_t)r;
		g = max * (float)tmp.g/255.0f;
		tmp.g = (uint8_t)g;
		b = max * (float)tmp.b/255.0f;
		tmp.b = (uint8_t)b;
		img[i] = tmp;
	}
}

/*
 * cancella un piano dell'immagine [plane = 'r' o 'g' o 'b'] fissato
 */
__global__ void deletePlane(AoS *img, const char plane, const int n) {
	unsigned int i = blockIdx.x * blockDim.x + threadIdx.x;
	if (i < n) {
		switch (plane) {
		case 'r':
			img[i].r = 0;
			break;
		case 'g':
			img[i].g = 0;
			break;
		case 'b':
			img[i].b = 0;
			break;
		}
	}
}

/*
 * setup device
 */
__global__ void warmup(AoS *img, const int max, const int n) {
	unsigned int i = blockIdx.x * blockDim.x + threadIdx.x;
	if (i < n) {
		float r,g,b;
		AoS tmp = img[i];
		r = max * (float)tmp.r/255.0f;
		tmp.r = (uint8_t)r;
		g = max * (float)tmp.g/255.0f;
		tmp.g = (uint8_t)g;
		b = max * (float)tmp.b/255.0f;
		tmp.b = (uint8_t)b;
		img[i] = tmp;
	}
}

/*
 * Legge da stdin quale kernel eseguire: 0 per rescaleImg, 1 per deletePlane
 */
int main(int argc, char **argv) {
	// set up device
	int dev = 0;
	cudaDeviceProp deviceProp;
	CHECK(cudaGetDeviceProperties(&deviceProp, dev));
	printf("%s test AoS at ", argv[0]);
	printf("device %d: %s \n", dev, deviceProp.name);
	CHECK(cudaSetDevice(dev));

	// scelta del kernel da eseguire
	int kernel = 0;
	if (argc > 1) kernel = atoi(argv[1]);

	// allocate host memory
	int n_elem = N;
	size_t nBytes = n_elem * sizeof(AoS);
	AoS *img = (AoS *)malloc(nBytes);
	AoS *new_img = (AoS *)malloc(nBytes);

	// initialize host array
	initialize(img, N);

	// allocate device memory
	AoS *d_img;
	CHECK(cudaMalloc((void**)&d_img, nBytes));

	// copy data from host to device
	CHECK(cudaMemcpy(d_img, img, nBytes, cudaMemcpyHostToDevice));

	// definizione max
	int max = 128;
	if (argc > 2) max = atoi(argv[2]);

	// configurazione per esecuzione
	dim3 block (blocksize, 1);
	dim3 grid  ((n_elem + block.x - 1) / block.x, 1);

	// kernel 1: warmup
	double iStart = seconds();
	warmup<<<1, 32>>>(d_img, max, 32);
	CHECK(cudaDeviceSynchronize());
	double iElaps = seconds() - iStart;
	printf("warmup<<< 1, 32 >>> elapsed %f sec\n",iElaps);
	CHECK(cudaGetLastError());

	// kernel 2 rescaleImg o deletePlane
	iStart = seconds();
	if (kernel == 0) {
		rescaleImg<<<grid, block>>>(d_img, max, n_elem);
		printf("rescaleImg <<< %3d, %3d >>> elapsed %f sec\n", grid.x, block.x, iElaps);
	}
	else {
		deletePlane<<<grid, block>>>(d_img, 'r', n_elem);
		printf("deletePlane <<< %3d, %3d >>> elapsed %f sec\n", grid.x, block.x, iElaps);
	}
	CHECK(cudaDeviceSynchronize());
	iElaps = seconds() - iStart;
	CHECK(cudaMemcpy(new_img, d_img, nBytes, cudaMemcpyDeviceToHost));
	CHECK(cudaGetLastError());

	//checkResult(img, new_img, n_elem);

	// free memories both host and device
	CHECK(cudaFree(d_img));
	free(img);
	free(new_img);

	// reset device
	CHECK(cudaDeviceReset());
	return EXIT_SUCCESS;
}

void initialize(AoS *img,  int size) {
	for (int i = 0; i < size; i++) {
		img[i].r = rand() % 256;
		img[i].g = rand() % 256;
		img[i].b = rand() % 256;
	}
	return;
}

void checkResult(AoS *img, AoS *new_img, int n_elem) {
	for (int i = 0; i < n_elem; i+=1000)
		printf("img[%d] = (%d,%d,%d) -- new_img[%d] = (%d,%d,%d)\n",
				i,img[i].r,img[i].g,img[i].b,i,new_img[i].r,new_img[i].g,new_img[i].b);
	return;
}



In [ ]:
# Compilazione ed esecuzione
!nvcc -arch=sm_37  struct/AoS.cu -o AoS
!./AoS

# ✅ Transpose

In [ ]:
%%writefile transpose/transpose.cu

#include <stdio.h>
#include "../../utils/common.h"

/*
 * Various memory access pattern optimizations applied to a matrix transpose kernel.
 */

#define BDIMX 16
#define BDIMY 16

void initialData(float *in,  const int size) {
  for (int i = 0; i < size; i++)
    in[i] = (float)( rand() & 0xFF ) / 10.0f; //100.0f;
  return;
}

void printData(float *in,  const int size) {
  for (int i = 0; i < size; i++)
    printf("%dth element: %f\n", i, in[i]);
  return;
}

void checkResult(float *hostRef, float *gpuRef, const int size, int showme) {
    double epsilon = 1.0E-8;
    bool match = 1;

    for (int i = 0; i < size; i++) {
        if (abs(hostRef[i] - gpuRef[i]) > epsilon) {
            match = 0;
            printf("different on %dth element: host %f gpu %f\n", i, hostRef[i],
                    gpuRef[i]);
            break;
        }

        if (showme && i > size / 2 && i < size / 2 + 5)
          printf("%dth element: host %f gpu %f\n",i,hostRef[i],gpuRef[i]);
    }

    if (!match)  printf("Arrays do not match.\n\n");
}

void transposeHost(float *out, float *in, const int nx, const int ny) {
  for( int iy = 0; iy < ny; ++iy)
    for( int ix = 0; ix < nx; ++ix)
      out[ix * ny + iy] = in[iy * nx + ix];
}

__global__ void warmup(float *out, float *in, const int nx, const int ny)
{
    unsigned int ix = blockDim.x * blockIdx.x + threadIdx.x;
    unsigned int iy = blockDim.y * blockIdx.y + threadIdx.y;

    if (ix < nx && iy < ny)
    {
        out[iy * nx + ix] = in[iy * nx + ix];
    }
}

// case 0 copy kernel: access data in rows
__global__ void copyRow(float *out, float *in, const int nx, const int ny)
{
    unsigned int ix = blockDim.x * blockIdx.x + threadIdx.x;
    unsigned int iy = blockDim.y * blockIdx.y + threadIdx.y;

    if (ix < nx && iy < ny)
    {
        out[iy * nx + ix] = in[iy * nx + ix];
    }
}

// case 1 copy kernel: access data in columns
__global__ void copyCol(float *out, float *in, const int nx, const int ny)
{
    unsigned int ix = blockDim.x * blockIdx.x + threadIdx.x;
    unsigned int iy = blockDim.y * blockIdx.y + threadIdx.y;

    if (ix < nx && iy < ny)
    {
        out[ix * ny + iy] = in[ix * ny + iy];
    }
}

// case 2 transpose kernel: read in rows and write in columns
__global__ void transposeNaiveRow(float *out, float *in, const int nx, const int ny) {
    unsigned int ix = blockDim.x * blockIdx.x + threadIdx.x;
    unsigned int iy = blockDim.y * blockIdx.y + threadIdx.y;

    if (ix < nx && iy < ny)
        out[ix * ny + iy] = in[iy * nx + ix];
}

// case 3 transpose kernel: read in columns and write in rows
__global__ void transposeNaiveCol(float *out, float *in, const int nx,
                                  const int ny)
{
    unsigned int ix = blockDim.x * blockIdx.x + threadIdx.x;
    unsigned int iy = blockDim.y * blockIdx.y + threadIdx.y;

    if (ix < nx && iy < ny)
    {
        out[iy * nx + ix] = in[ix * ny + iy];
    }
}

// case 4 transpose kernel: read in rows and write in columns + unroll 4 blocks
__global__ void transposeUnroll4Row(float *out, float *in, const int nx,
                                    const int ny)
{
    unsigned int ix = blockDim.x * blockIdx.x * 4 + threadIdx.x;
    unsigned int iy = blockDim.y * blockIdx.y + threadIdx.y;

    unsigned int ti = iy * nx + ix; // access in rows
    unsigned int to = ix * ny + iy; // access in columns

    if (ix + 3 * blockDim.x < nx && iy < ny)
    {
        out[to]                   = in[ti];
        out[to + ny * blockDim.x]   = in[ti + blockDim.x];
        out[to + ny * 2 * blockDim.x] = in[ti + 2 * blockDim.x];
        out[to + ny * 3 * blockDim.x] = in[ti + 3 * blockDim.x];
    }
}

// case 5 transpose kernel: read in columns and write in rows + unroll 4 blocks
__global__ void transposeUnroll4Col(float *out, float *in, const int nx,
                                    const int ny)
{
    unsigned int ix = blockDim.x * blockIdx.x * 4 + threadIdx.x;
    unsigned int iy = blockDim.y * blockIdx.y + threadIdx.y;

    unsigned int ti = iy * nx + ix; // access in rows
    unsigned int to = ix * ny + iy; // access in columns

    if (ix + 3 * blockDim.x < nx && iy < ny)
    {
        out[ti]                = in[to];
        out[ti +   blockDim.x] = in[to +   blockDim.x * ny];
        out[ti + 2 * blockDim.x] = in[to + 2 * blockDim.x * ny];
        out[ti + 3 * blockDim.x] = in[to + 3 * blockDim.x * ny];
    }
}

/*
 * case 6 :  transpose kernel: read in rows and write in colunms + diagonal
 * coordinate transform
 */
__global__ void transposeDiagonalRow(float *out, float *in, const int nx,
                                     const int ny)
{
    unsigned int blk_y = blockIdx.x;
    unsigned int blk_x = (blockIdx.x + blockIdx.y) % gridDim.x;

    unsigned int ix = blockDim.x * blk_x + threadIdx.x;
    unsigned int iy = blockDim.y * blk_y + threadIdx.y;

    if (ix < nx && iy < ny)
    {
        out[ix * ny + iy] = in[iy * nx + ix];
    }
}

/*
 * case 7 :  transpose kernel: read in columns and write in row + diagonal
 * coordinate transform.
 */
__global__ void transposeDiagonalCol(float *out, float *in, const int nx,
                                     const int ny)
{
    unsigned int blk_y = blockIdx.x;
    unsigned int blk_x = (blockIdx.x + blockIdx.y) % gridDim.x;

    unsigned int ix = blockDim.x * blk_x + threadIdx.x;
    unsigned int iy = blockDim.y * blk_y + threadIdx.y;

    if (ix < nx && iy < ny)
    {
        out[iy * nx + ix] = in[ix * ny + iy];
    }
}

// main functions
int main(int argc, char **argv)
{
    // set up device
    int dev = 0;
    cudaDeviceProp deviceProp;
    CHECK(cudaGetDeviceProperties(&deviceProp, dev));
    printf("%s starting transpose at ", argv[0]);
    printf("device %d: %s ", dev, deviceProp.name);
    CHECK(cudaSetDevice(dev));

    // set up array size 2048
    int nx = 1 << 12;
    int ny = 1 << 12;

    // select a kernel and block size
    int iKernel = 0;
    int blockx = 16;
    int blocky = 16;

    if (argc > 1) iKernel = atoi(argv[1]);

    if (argc > 2) blockx  = atoi(argv[2]);

    if (argc > 3) blocky  = atoi(argv[3]);

    if (argc > 4) nx  = atoi(argv[4]);

    if (argc > 5) ny  = atoi(argv[5]);

    printf(" with matrix nx %d ny %d with kernel %d\n", nx, ny, iKernel);
    size_t nBytes = nx * ny * sizeof(float);

    // execution configuration
    dim3 block (blockx, blocky);
    dim3 grid  ((nx + block.x - 1) / block.x, (ny + block.y - 1) / block.y);

    // allocate host memory
    float *h_A = (float *)malloc(nBytes);
    float *hostRef = (float *)malloc(nBytes);
    float *gpuRef  = (float *)malloc(nBytes);

    // initialize host array
    initialData(h_A, nx * ny);

    // transpose at host side
    transposeHost(hostRef, h_A, nx, ny);

    // allocate device memory
    float *d_A, *d_C;
    CHECK(cudaMalloc((float**)&d_A, nBytes));
    CHECK(cudaMalloc((float**)&d_C, nBytes));

    // copy data from host to device
    CHECK(cudaMemcpy(d_A, h_A, nBytes, cudaMemcpyHostToDevice));

    // warmup to avoide startup overhead
    double iStart = seconds();
    warmup<<<grid, block>>>(d_C, d_A, nx, ny);
    CHECK(cudaDeviceSynchronize());
    double iElaps = seconds() - iStart;
    printf("warmup         elapsed %f sec\n", iElaps);
    CHECK(cudaGetLastError());

    // kernel pointer and descriptor
    void (*kernel)(float *, float *, int, int);
    const char *kernelName;

    // set up kernel
    switch (iKernel)
    {
    case 0:
        kernel = &copyRow;
        kernelName = "CopyRow       ";
        break;

    case 1:
        kernel = &copyCol;
        kernelName = "CopyCol       ";
        break;

    case 2:
        kernel = &transposeNaiveRow;
        kernelName = "NaiveRow      ";
        break;

    case 3:
        kernel = &transposeNaiveCol;
        kernelName = "NaiveCol      ";
        break;

    case 4:
        kernel = &transposeUnroll4Row;
        kernelName = "Unroll4Row    ";
        grid.x = (nx + block.x * 4 - 1) / (block.x * 4);
        break;

    case 5:
        kernel = &transposeUnroll4Col;
        kernelName = "Unroll4Col    ";
        grid.x = (nx + block.x * 4 - 1) / (block.x * 4);
        break;

    case 6:
        kernel = &transposeDiagonalRow;
        kernelName = "DiagonalRow   ";
        break;

    case 7:
        kernel = &transposeDiagonalCol;
        kernelName = "DiagonalCol   ";
        break;
    }

    // run kernel
    iStart = seconds();
    kernel<<<grid, block>>>(d_C, d_A, nx, ny);
    CHECK(cudaDeviceSynchronize());
    iElaps = seconds() - iStart;

    // calculate effective_bandwidth
    float ibnd = 2 * nx * ny * sizeof(float) / 1e9 / iElaps;
    printf("%s elapsed %f sec <<< grid (%d,%d) block (%d,%d)>>> effective "
           "bandwidth %f GB\n", kernelName, iElaps, grid.x, grid.y, block.x,
           block.y, ibnd);
    CHECK(cudaGetLastError());

    // check kernel results
    if (iKernel > 1)
    {
        CHECK(cudaMemcpy(gpuRef, d_C, nBytes, cudaMemcpyDeviceToHost));
        checkResult(hostRef, gpuRef, nx * ny, 1);
    }

    // free host and device memory
    CHECK(cudaFree(d_A));
    CHECK(cudaFree(d_C));
    free(h_A);
    free(hostRef);
    free(gpuRef);

    // reset device
    CHECK(cudaDeviceReset());
    return EXIT_SUCCESS;
}


In [ ]:
# Compilazione ed esecuzione
!nvcc -arch=sm_37  transpose/transpose.cu -o transp
!./transp 0

In [ ]:
!./transp 1

In [ ]:
!./transp 2

In [ ]:
!./transp 3

In [ ]:
!./transp 4

In [ ]:
!./transp 5

In [ ]:
!./transp 6

In [ ]:
!./transp 7